In [16]:
url = "https://www.tanitjobs.com/jobs/?searchId=1731624656.9136&action=search&page=1"

In [20]:
import requests
import re
from bs4 import BeautifulSoup

def get_posts_links(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'html.parser')
    print(soup.prettify())
    # posts = soup.find_all('div', class_='media-body')
    posts = soup.find('div', class_='media-body')
    print(posts)

In [ ]:
import httpx
from parsel import Selector

response = httpx.get("some web page with cloudflare challenge URL")
selector = Selector(response.text)

# in ScrapFly becomes this 👇
from scrapfly import ScrapeConfig, ScrapflyClient

# replaces your HTTP client (httpx in this case)
scrapfly = ScrapflyClient(key="Your ScrapFly API key")

response = scrapfly.scrape(ScrapeConfig(
    url="web page URL",
    asp=True, # enable the anti scraping protection to bypass blocking
    country="US", # set the proxy location to a specfic country
    proxy_pool="public_residential_pool", # select a proxy pool
    render_js=True # enable rendering JavaScript (like headless browsers) to scrape dynamic content if needed
))

# use the built in Parsel selector
selector = response.selector
# access the HTML content
html = response.scrape_result['content']

TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [37]:
import asyncio
from flare_bypasser import BrowserWrapper, Request, Response
from flare_bypasser import FlareSolverr


# Define the MyClickCommandProcessor class (same as before)
class MyClickCommandProcessor(BaseCommandProcessor):
    async def preprocess_command(self, req: Request, driver: BrowserWrapper) -> Request:
        if "target_element" not in req.params:
            raise ValueError("Missing target_element parameter")
        return req

    async def process_command(self, res: Response, req: Request, driver: BrowserWrapper) -> Response:
        # Get the driver instance
        nodriver_tab = driver.get_driver()
        
        # Get the current DOM content
        dom = await nodriver_tab.get_content()
        
        # Find all 'input[type=submit]' elements
        els = await nodriver_tab.select_all('input[type=submit]')
        
        if not els:
            # If no 'input[type=submit]' elements are found, raise an exception
            raise Exception("MyClickCommandProcessor: no input for click: " + str(dom))
        
        # Click on the first 'input[type=submit]' element found
        await els[0].click()
        
        # Wait for the content to update after the click
        res.response = await nodriver_tab.get_content()
        
        # Return the updated response
        return res


# Define a function to run the automation
async def run_automation(url):
    # Initialize FlareSolverr and BrowserWrapper
    flare_solverr = FlareSolverr()
    driver = await flare_solverr.get_driver()

    # Create the request with the given URL
    req = Request(url=url)

    # Create an empty response to be updated
    res = Response()

    # Create the custom command processor
    command_processor = MyClickCommandProcessor()

    # Apply the command processor to click the 'VOIR PLUS' button
    await command_processor.process_command(res, req, driver)

    # After processing, print the updated page content
    print(res.response)

    # Close the driver
    await driver.close()


# URL to visit
url = "https://www.tanitjobs.com/jobs/?searchId=1731624656.9136&action=search&page=1"

# Run the automation asynchronously
asyncio.run(run_automation(url))

ImportError: cannot import name 'FlareSolverr' from 'flare_bypasser' (d:\INSAT\RT4\Semestre 1\ML\mini-project\JobTN-pipeline\.venv\lib\site-packages\flare_bypasser\__init__.py)